In [15]:
import pandas as pd
import numpy as np
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

import plotly.express as px
import plotly.graph_objects as go

ModuleNotFoundError: No module named 'mlxtend'

In [14]:
df = pd.read_csv("../data/processed/cleaned_data.csv", index_col=0)
print(df.shape)
df.head(10)

(445, 23)


,CODE,age.diag,age.menarc,age.menop,education,marital,gravity,lactaton,fpregnancy,infertility,...,hrt,smoking,personal.Hx,Family,agef,weight,height,lifeevent,stage,G
1,86002380,41,12,47,3,2,1,27,20,0,...,0,0,1,4,38,69,158,0,3a,2
15,3,53,16,50,2,2,1,52,19,0,...,0,0,1,0,44,70,155,0,1,2
22,6000094,41,14,47,3,2,1,0,21,0,...,0,0,1,0,44,63,157,0,3c,2
26,6000178,40,14,47,3,2,1,19,22,0,...,0,0,0,0,44,63,154,0,2a,1
28,6000211,38,11,47,3,2,1,18,22,0,...,0,0,1,1,44,53,150,0,2b,2
33,6000377,32,13,47,4,2,2,0,21,0,...,0,0,0,0,44,60,166,0,3c,2
38,6000618,27,15,47,3,2,1,9,19,0,...,0,0,1,0,44,68,167,0,3c,2
40,6000712,46,15,45,2,2,1,0,18,0,...,0,0,1,3,44,68,158,0,3a,2
43,6000797,48,13,48,4,3,1,0,34,0,...,0,1,1,0,44,75,160,0,3a,3
45,6000801,52,16,52,3,2,2,0,21,0,...,0,0,1,0,44,59,153,0,2a,3
